In [100]:
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_test.drop('ID', axis=1, inplace=True)

df_target = df['SUBCLASS']
data2 = df.drop(['ID'],axis=1)
data = df.drop(['ID', 'SUBCLASS'],axis=1)
disease_list = df.SUBCLASS.unique()

In [101]:
f = lambda x : 0 if x == 'WT' else 1
data = data.map(f)
#WT 모든 컬럼이 의미 없는 값인 거 치환하고 시작하기
zerosl = list(data[(data == 0).all(axis=1)].index)
df.loc[zerosl, 'SUBCLASS'] = 'THYM'
data = df.copy().drop(['ID', 'SUBCLASS'], axis=1)
total = pd.concat([data, df_test])
f = lambda x : '' if x == 'WT' else x
total = total.map(f)
total


,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,,,L305L,,N252I N251I,G679V,,
2544,,,,,,,,,R1517H,,...,,,,,,,,,,


In [111]:
df[df['HDAC3'].apply(lambda x : x != 'WT')]

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
58,TRAIN_0058,STES,WT,WT,WT,WT,WT,WT,G1432R C1201Y,WT,...,E2046fs,WT,WT,WT,WT,WT,WT,WT,WT,WT
158,TRAIN_0158,OV,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
322,TRAIN_0322,STES,M713I,Q456fs,WT,WT,R220R A304T,L2012P,WT,WT,...,WT,WT,WT,WT,D926G,WT,WT,WT,WT,WT
406,TRAIN_0406,UCEC,WT,S205N,T8M,WT,WT,R2014K S264I,WT,F377F,...,G499G E1507K S2056Y I2571S,WT,WT,WT,F380F,WT,WT,WT,WT,WT
495,TRAIN_0495,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,V301V Q14Q,WT,WT,WT,WT,WT,WT,WT,WT
520,TRAIN_0520,UCEC,R1297R,WT,WT,WT,M155I,WT,WT,R106*,...,R713H A862S,WT,W193*,WT,R1037C,A385V,WT,E336D,WT,WT
611,TRAIN_0611,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
961,TRAIN_0961,STES,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1016,TRAIN_1016,LUSC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1274,TRAIN_1274,STES,WT,WT,WT,WT,WT,Y857H N2133fs N2133fs,A1996A S1686S,R1550W N568N,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [107]:
df.iloc[58]

ID          TRAIN_0058
SUBCLASS          STES
A2M                 WT
AAAS                WT
AADAT               WT
               ...    
ZNRF4               WT
ZPBP                WT
ZW10                WT
ZWINT               WT
ZYX                 WT
Name: 58, Length: 4386, dtype: object

In [105]:
df.iloc[4766]

ID          TRAIN_4766
SUBCLASS          LUAD
A2M             R1034S
AAAS                WT
AADAT               WT
               ...    
ZNRF4               WT
ZPBP                WT
ZW10                WT
ZWINT               WT
ZYX                 WT
Name: 4766, Length: 4386, dtype: object

In [106]:
df.iloc[5214]

ID          TRAIN_5214
SUBCLASS          HNSC
A2M                 WT
AAAS                WT
AADAT               WT
               ...    
ZNRF4               WT
ZPBP                WT
ZW10                WT
ZWINT               WT
ZYX                 WT
Name: 5214, Length: 4386, dtype: object

In [102]:
#nrows = df.shape[0]
#df = total[:nrows]

#df_test = total[nrows:]

In [103]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>5]['index'].values
        print(d, len(gene_list))
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

In [ ]:
df_target = df['SUBCLASS']

data = df.loc[:, list(glist)]
df_test = df_test.loc[:, list(glist)]


In [3]:
dd_list = ['BRCA', 'KIPAN' , 'GBMLGG', 'STES']
#too many 변이, ROWs DELETE
for d in disease_list:
    td = df[df['SUBCLASS']==d].apply(lambda x : x!='WT').agg(sum , axis=1).sort_values(ascending=False) >= 800 #706
    if td[td].sum()> 0:
        print(d, list(td[td].index))
        df.drop(list(td[td].index), inplace=True)


data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns=['ID'], inplace=True)
data

(6178, 4386)

In [5]:
def get_count_df(df, d, th):
    count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count').sort_values(by='count',ascending=False)
    gl = count_df[count_df['count']>th][1:]['index']
    return gl

from collections import defaultdict
expert_dict = defaultdict()

df_ = data.copy()

df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
for d in disease_list:
    all_g = set(df[df.SUBCLASS==d].columns)
    gl = get_count_df(df_, d, 1)
    try:
        g_expert = pd.read_csv('./reference_data/'+d+'.csv')
        aa = set(g_expert['g.name'])        
        
    except FileNotFoundError:
        aa = set(get_count_df(df_, d, 5))
        
    expdict = all_g.intersection(aa)
    expert_dict[d] = expdict
    print(d,  len(list(gl)), len(aa), len(expdict))

KIPAN 1873 699 307
SARC 513 69 38
SKCM 3407 69 31
KIRC 1050 132 69
GBMLGG 1131 181 117
STES 3566 69 38
BRCA 2369 1004 497
THCA 197 58 34
LIHC 1161 179 104
HNSC 1948 55 37
PAAD 362 628 287
OV 818 317 133
PRAD 370 611 311
UCEC 2203 89 54
LAML 49 1007 493
COAD 2519 557 278
ACC 555 38 23
LGG 288 111 53
LUSC 2399 629 246
LUAD 2116 467 467
CESC 1652 42 22
PCPG 72 3 3
THYM 30 19 10
BLCA 1774 121 70
TGCT 430 23 9
DLBC 553 9 4


In [26]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('disease').reset_index(name='counts')
diseaseList

,disease,counts
0,BRCA,781
1,KIPAN,511
2,GBMLGG,459
3,STES,377
4,KIRC,332
5,THCA,306
6,SKCM,276
7,PRAD,262
8,OV,253
9,LGG,226


In [27]:
class_dict = defaultdict()
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>5]['index'].values
        print(d, len(gene_list))
        class_dict[d] = gene_list
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

NameError: name 'defaultdict' is not defined

In [8]:
selected_genes = set()
for d in disease_list :
    expd = expert_dict[d]
    selected_genes.update(list(expd))
len(selected_genes)

2051

In [63]:
selected_genes = set()
for d in disease_list:
    tdf = df [ df['SUBCLASS'].isin([d] )]
    total_len = tdf.shape[0]
    
    tdf = tdf.apply(lambda x : x !='WT').agg(sum)
    tdf = tdf.iloc[2:].reset_index(name='count').sort_values(by='count', ascending=False)
    
    rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()
    
    print(d, len(rlist))
    
    selected_genes.update(rlist)
#selected_genes = list(selected_genes)
len(selected_genes)

KIPAN 286
SARC 31
SKCM 2112
KIRC 68
GBMLGG 147
STES 2034
BRCA 691
THCA 4
LIHC 106
HNSC 334
PAAD 22
OV 47
PRAD 16
UCEC 2006
LAML 9
COAD 936
ACC 51
LGG 16
LUSC 587
LUAD 468
CESC 175
PCPG 3
THYM 1
BLCA 209
TGCT 14
DLBC 17


2925

In [29]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [6]:
#selected_genes.remove("SUBCLASS")

In [32]:
df_target = df['SUBCLASS']

data = df.loc[:, list(selected_genes)]
df_test = df_test.loc[:, list(selected_genes)]


In [29]:
df_target = df['SUBCLASS']

data = df.loc[:, list(glist)]
df_test = df_test.loc[:, list(glist)]


In [66]:
total = pd.concat([data, df_test] ) 
f = lambda x : '' if x == 'WT' else x
total = total.map(f)
total

,CSAD,COL6A2,GART,LEPR,CIT,KRT15,MLPH,PCP4,CDH3,CLSTN3,...,GEM,AKR1C3,CTH,MAPT,KALRN,HOMER1,NELFB,CRMP1,MCM3,MLF1
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,80,23,108,191,0,0,0,40,0,...,0,0,22,0,257,0,44,14,0,0
2544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
total

,CSAD,COL6A2,GART,LEPR,CIT,KRT15,MLPH,PCP4,CDH3,CLSTN3,...,GEM,AKR1C3,CTH,MAPT,KALRN,HOMER1,NELFB,CRMP1,MCM3,MLF1
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,80,23,108,191,0,0,0,40,0,...,0,0,22,0,257,0,44,14,0,0
2544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
total = total.loc[:, list(selected_genes)]
total

,CSAD,COL6A2,GART,LEPR,CIT,KRT15,MLPH,PCP4,CDH3,CLSTN3,...,GEM,AKR1C3,CTH,MAPT,KALRN,HOMER1,NELFB,CRMP1,MCM3,MLF1
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,G436C,E191X,P1143S,R1989Q R2031Q,,,,G824C G769C,,...,,,L100I,,R1115C R1087C R1128C R1124C R1137C,,M402I,A386T A502T A382T A388T,,
2544,,,,,,,,,,,...,,,,,,,,,,


## apply LabelEncoder 

In [41]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for c in total.columns:
    total[c] = encoder.fit_transform(total[c])

nRows = df.shape[0]
X = total[:nRows]
df_test = total[nRows:]
y = df_target



encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)
y


array([ 8, 19, 20, ...,  4, 22, 20])

In [42]:
y.shape[0]

6201

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)#shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 2925) (1861, 2925)


#Multi-class classification

In [44]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [45]:
params = {
    'tree_method': 'approx',
    'objective':  'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.60217	valid-mlogloss:2.70251
[2]	train-mlogloss:2.20521	valid-mlogloss:2.44441
[4]	train-mlogloss:1.97445	valid-mlogloss:2.32559
[6]	train-mlogloss:1.81261	valid-mlogloss:2.25351
[8]	train-mlogloss:1.69204	valid-mlogloss:2.20603
[10]	train-mlogloss:1.59626	valid-mlogloss:2.17624
[12]	train-mlogloss:1.51884	valid-mlogloss:2.15811
[14]	train-mlogloss:1.45555	valid-mlogloss:2.14265
[16]	train-mlogloss:1.40093	valid-mlogloss:2.13108
[18]	train-mlogloss:1.35266	valid-mlogloss:2.12530
[20]	train-mlogloss:1.30977	valid-mlogloss:2.12288
[22]	train-mlogloss:1.27097	valid-mlogloss:2.12274
[24]	train-mlogloss:1.23729	valid-mlogloss:2.12029
[26]	train-mlogloss:1.20582	valid-mlogloss:2.11868
[28]	train-mlogloss:1.17506	valid-mlogloss:2.12077
[30]	train-mlogloss:1.14762	valid-mlogloss:2.12286
[31]	train-mlogloss:1.13416	valid-mlogloss:2.12264


In [46]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    #'learning_rate': 0.05
}
num_boost_round = 1000

clf = xgb.XGBClassifier( n_estimators = 100,        # 부스팅 단계 수
                        max_depth = 8,             # 개별 트리의 최대 깊이
                        learning_rate = 0.1,       # 학습률, 낮출수록 느리게 학습하지만 성능 개선 가능
                        subsample=0.8,           # 각 부스팅 단계에서 사용할 샘플의 비율
                        #scale_pos_weight = class_weight_dict,  # 클래스별 가중치
                        colsample_bytree=0.8,    # 각 트리에서 사용할 특성의 비율
                        random_state = 42,         # 랜덤 시드
                        n_jobs = -1,             # 모든 CPU 코어 사용,
                        early_stopping_rounds = 5,
                        **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.06418	validation_1-mlogloss:3.09068
[2]	validation_0-mlogloss:2.76746	validation_1-mlogloss:2.85798
[4]	validation_0-mlogloss:2.58628	validation_1-mlogloss:2.73001
[6]	validation_0-mlogloss:2.42736	validation_1-mlogloss:2.62058
[8]	validation_0-mlogloss:2.29623	validation_1-mlogloss:2.53518
[10]	validation_0-mlogloss:2.18369	validation_1-mlogloss:2.46898
[12]	validation_0-mlogloss:2.08942	validation_1-mlogloss:2.41622
[14]	validation_0-mlogloss:2.00595	validation_1-mlogloss:2.37319
[16]	validation_0-mlogloss:1.93268	validation_1-mlogloss:2.33409
[18]	validation_0-mlogloss:1.86876	validation_1-mlogloss:2.30439
[20]	validation_0-mlogloss:1.80860	validation_1-mlogloss:2.27785
[22]	validation_0-mlogloss:1.75588	validation_1-mlogloss:2.25824
[24]	validation_0-mlogloss:1.70611	validation_1-mlogloss:2.23881
[26]	validation_0-mlogloss:1.66001	validation_1-mlogloss:2.22151
[28]	validation_0-mlogloss:1.61674	validation_1-mlogloss:2.20459
[30]	validation_0-mlogloss:1.5

In [47]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1861,), (1861,), (1861, 26))

In [48]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.3417517463729178

In [21]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.33117583603020495

In [49]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.74      0.83        23
           1       0.56      0.14      0.23        35
           2       0.28      0.66      0.40       236
           3       0.15      0.05      0.08        39
           4       0.65      0.62      0.63        63
           5       1.00      0.19      0.32        16
           6       0.33      0.29      0.31       149
           7       0.30      0.23      0.26        60
           8       0.25      0.34      0.28       159
           9       0.07      0.05      0.06       104
          10       0.71      0.35      0.47        43
          11       0.24      0.35      0.29        57
          12       0.20      0.10      0.14        48
          13       0.16      0.05      0.08        59
          14       0.32      0.12      0.18        49
          15       0.29      0.43      0.35        93
          16       0.06      0.04      0.05        28
          17       0.28    

In [50]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.859593035325779

In [51]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.859593035325779

## Create Result DataFrame

In [52]:
clf.predict(df_test)

array([21, 25,  2, ..., 25, 10,  6])

In [55]:
res = clf.predict(df_test)
res = encoder2.inverse_transform(res)
df_test1 = pd.read_csv('../input/test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,HNSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,UCEC
2544,TEST_2544,LAML


In [56]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      617
COAD      519
KIPAN     232
STES      161
OV        138
GBMLGG    121
UCEC       85
SKCM       83
THCA       82
LGG        63
HNSC       54
KIRC       53
LIHC       46
LAML       41
TGCT       40
PRAD       38
CESC       34
PCPG       26
BLCA       24
LUSC       23
SARC       16
LUAD       16
ACC        14
PAAD       12
THYM        5
DLBC        3
Name: count, dtype: int64

In [57]:
res_df.to_csv('../res/res-0.csv',index = False)

## 01 Update Result with Zeros []

In [58]:
zerosl = list(df_test[(df_test == 0).all(axis=1)].index)
res_df.loc[zerosl, 'SUBCLASS'] = 'THYM'

In [59]:
res_df

,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,HNSC
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,UCEC
2544,TEST_2544,LAML


In [60]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      585
COAD      519
KIPAN     232
STES      161
OV        138
GBMLGG    121
UCEC       85
SKCM       83
THCA       82
LGG        63
HNSC       54
KIRC       53
LIHC       46
LAML       41
TGCT       40
PRAD       38
THYM       37
CESC       34
PCPG       26
BLCA       24
LUSC       23
SARC       16
LUAD       16
ACC        14
PAAD       12
DLBC        3
Name: count, dtype: int64

In [61]:
res_df.loc[180]

ID          TEST_0180
SUBCLASS         THYM
Name: 180, dtype: object

## SAVE res_df with zero step

In [62]:
res_df.to_csv('../res/step-1.csv',index = False)

In [35]:
############## <5 Finding SubSet

In [59]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns='ID',inplace=True)

total = pd.concat([data, df_test])

f = lambda x : 0 if x == 'WT' else 1
total = total.map(f)

data = total[:6201]
df_test = total[6201:]


In [60]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#하나라도 1인  

# 딱 1개만 1인 행 필터링
one_1_rows = df_test[df_test.sum(axis=1) == 1]

# 각 행에서 1이 위치한 열의 이름과 인덱스를 구하는 코드
result = [(index, row.idxmax()) for index, row in one_1_rows.iterrows()]
result

[(8, 'SOX9'),
 (90, 'BRAF'),
 (110, 'LIG1'),
 (131, 'IDH2'),
 (170, 'PLK4'),
 (192, 'ZNF365'),
 (260, 'BRAF'),
 (352, 'PRKDC'),
 (366, 'VHL'),
 (390, 'DOCK2'),
 (435, 'CDCP1'),
 (619, 'GFPT2'),
 (655, 'CTNNA1'),
 (672, 'CD46'),
 (693, 'HRAS'),
 (701, 'HRAS'),
 (798, 'CCNL1'),
 (880, 'GNAO1'),
 (898, 'ATXN1'),
 (1023, 'MYH1'),
 (1034, 'CDKN2A'),
 (1071, 'VHL'),
 (1078, 'CACNA1A'),
 (1118, 'ACE'),
 (1202, 'VHL'),
 (1263, 'BRAF'),
 (1287, 'TG'),
 (1408, 'VHL'),
 (1426, 'VHL'),
 (1437, 'SREBF2'),
 (1560, 'CACNA1A'),
 (1690, 'LRPPRC'),
 (1761, 'VHL'),
 (1804, 'RAD21'),
 (1830, 'LIG1'),
 (1833, 'VHL'),
 (1851, 'NPM1'),
 (1879, 'VHL'),
 (1884, 'KMT2D'),
 (1979, 'NPM1'),
 (2056, 'VHL'),
 (2086, 'VHL'),
 (2114, 'BRCA2'),
 (2150, 'NPM1'),
 (2218, 'TP53'),
 (2383, 'NPM1'),
 (2447, 'CDH1'),
 (2512, 'SUCLG1')]

In [ ]:
one_VHL_l=[]
one_BRAF_l = []
one_NPM1_l = []
one_LIG1_l = []
one_s = []
res_df1 = res_df.copy()

for i, c in result:
    print(i, c)
    if c =='VHL': 
        one_VHL_l.append(i)
        # 해당 컬럼에서 SUBCLASS값들의 분포를 구함
        q = ['VHL']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)] 
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #KIRC
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
    elif c=='BRAF': 
        one_BRAF_l.append(i)
        q = ['BRAF']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #maybe 1
        res_df1.loc[one_BRAF_l,'SUBCLASS'] = rs
    elif c=='NPM1' : 
        one_NPM1_l.append(i)
        q = ['NPM1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_NPM1_l,'SUBCLASS'] = rs
    elif c=='LIG1' : 
        one_LIG1_l.append(i)
        q = ['LIG1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_LIG1_l,'SUBCLASS'] = rs
    elif c=='HRAS': 
        one_s.append(i)
        q = ['HRAS']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS#0 not 1
        res_df1.loc[one_s,'SUBCLASS'] = rs
    #else:
        ###one_s.append(i)
        #row_no = i
        q = [c]
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
        res_df1.loc[i]['SUBCLASS'] = rs
    

res_df1.to_csv('res-step2-ones-2-else2.csv',index = False)
    

In [ ]:
############# optional code

In [ ]:
## TEST 데이터에서 1개의 컬럼만 1인 경우 찾기
one_1_rows = df_test[df_test.sum(axis=1) == 1]
# 각 열(column)별로 1이 있는 인덱스 리스트 생성
result_dict = {}
for column in one_1_rows.columns:
    indices = one_1_rows[one_1_rows[column] == 1].index.tolist()
    if indices:  # 인덱스가 비어있지 않으면 추가
        result_dict[column] = indices

# 딕셔너리를 DataFrame으로 변환
result_df = pd.DataFrame(list(result_dict.items()), columns=['gene', 'index_list'])
result_df


In [ ]:
for q in result_df.gene:
    print(q)
    rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
    rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    if rs.shape[0] > 1:
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs.loc[1].SUBCLASS
    elif rs.shape[0] ==1:
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs.loc[0].SUBCLASS
    print(rs)
res_df1.to_csv('res-step2-ones-3.csv',index = False)

In [ ]:
q = 'VHL'
rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
#res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
print(rs)

In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## 1이 2개 이상 있는 행 필터링
one_2_rows = df_test[df_test.sum(axis=1) == 2]

# 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 출력
print(result)


## Update Res_df with 2 / 3 / 4 / 5

In [ ]:
data = data.loc[:, list(glist)]
df_test = df_test.loc[:, list(glist)]

In [63]:
## TEST DATA에서 1이 n개 이상 있는 행 필터링
def find_test_n_ones(n):
    one_2_rows = df_test[df_test.sum(axis=1) == n]
    # 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
    result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]
    return result
print(find_test_n_ones(3))

[(11, ['HK1', 'NPM1', 'VIPR1']), (22, ['ALMS1', 'ATRX', 'IDH1']), (145, ['BRAF', 'CDC42EP4', 'PPRC1']), (241, ['NCOA6', 'PABPC1', 'RGN']), (320, ['AIFM1', 'THRB', 'TUBGCP6']), (333, ['MMP2', 'PTEN', 'TRIM58']), (398, ['ABCG4', 'LCT', 'SCN9A']), (456, ['IDH2', 'NOTCH1', 'RUNX1']), (490, ['BRAF', 'FETUB', 'LAMC1']), (491, ['GATA3', 'NRCAM', 'PLCG1']), (492, ['HSP90AB1', 'MKNK1', 'PTEN']), (586, ['BRAF', 'GOT2', 'UQCRC1']), (656, ['GPD2', 'HRAS', 'PUS1']), (669, ['ANXA5', 'MKI67', 'RET']), (736, ['CCND3', 'CD59', 'HNF4A']), (812, ['AHNAK', 'DNAJB8', 'SUN2']), (852, ['NRIP1', 'PIK3CA', 'SLC30A1']), (856, ['ATRN', 'FLII', 'PLCG1']), (897, ['FANCC', 'SFRP1', 'SPTB']), (935, ['AZGP1', 'BRAF', 'THBS1']), (942, ['BUB1', 'HAO2', 'PTPRE']), (954, ['KRT13', 'NPM1', 'RREB1']), (978, ['ARFGEF1', 'FZD5', 'TP53']), (1016, ['AP3B1', 'IRS1', 'TMPRSS2']), (1022, ['GAD1', 'IMMT', 'PCYT2']), (1089, ['ATRX', 'CA12', 'GAA']), (1102, ['CREBBP', 'MYBPC3', 'SPTA1']), (1139, ['CHEK2', 'PIK3CG', 'TAF1C']), (1197,

In [68]:
for ns in range (2,5):
    result_n = find_test_n_ones(ns )

    for i, qSet in result_n:
        #print(i, qSet)
        #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
        rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
        rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
        
        if rs.shape[0] > 0:
            rs = rs.loc[0].SUBCLASS
            print(rs, res_df.loc[i]['SUBCLASS'])
            #res_df2.loc[i,'SUBCLASS'] = rs
    


LGG BRCA
LGG GBMLGG
GBMLGG GBMLGG
KIRC UCEC
KIPAN PCPG
KIPAN SARC
LGG LGG
GBMLGG GBMLGG
GBMLGG GBMLGG
LGG LGG
LGG LGG
GBMLGG LGG
LGG LAML
GBMLGG GBMLGG
LGG LGG
LGG GBMLGG
LGG LGG
LGG LGG
GBMLGG GBMLGG


In [ ]:
print(find_test_n_ones(4))

In [ ]:
print(find_test_n_ones(5))


In [ ]:
df_test

In [ ]:
# res 값 수정, 
# 문서 2,3,4,5
res_df2 = res_df1.copy()
result_n = find_test_n_ones(2)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1:
            try:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count')#.loc[].SUBCLASS## 0->1
                print(rs)
                #res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS## 0->1
                res_df2.loc[i,'SUBCLASS'] = rs
                print(rs)
            
#res_df2.to_csv('res2-step3-two.csv',index = False)

In [ ]:
result_n = find_test_n_ones(3)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    #else: 
    #    #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
    #    rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
    #    if rows_with_all_ones.shape[0] > 0 : 
    #        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
    #        print(rs)
    #        res_df2.loc[i,'SUBCLASS'] = rs


In [55]:
res_df2.to_csv('res-step4-three.csv',index = False)

NameError: name 'find_test_n_ones' is not defined

In [48]:
res_df2.to_csv('res-step5-four.csv',index = False)

In [ ]:
result_n = find_test_n_ones(5)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1 : 
            try: 
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs


In [58]:
res_df2.to_csv('res-step6-five.csv',index = False)